# Analysis

This file contains all the code for analysing the data

In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import glob
import re

## Minimum/ Maximum energy consumed in 15 mins per season


In [ ]:
# Function to extract numeric part from file name
season="spring"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv')
file_paths = sorted(file_paths, key=extract_numeric_part)
print(file_paths[:2])

In [ ]:
min_max_df= pl.DataFrame()

for i,file_path in enumerate(file_paths):
    #print(i)
    df=pl.read_csv(file_path)
    df1=pl.DataFrame({"min_values": df.select(pl.min("value_kwh")),
                      "max_values": df.select(pl.max("value_kwh")),
                      "mean_values": df.select(pl.mean("value_kwh"))})
    min_max_df=pl.concat([min_max_df,df1])
#print(min_max_df)
min_max_df = min_max_df.with_row_index()
min_max_df = min_max_df.rename({"index" : "House_no"})
print(min_max_df)

## Total Household Consumption

In [ ]:
# Function to extract numeric part from file name
season="winter"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv')
file_paths = sorted(file_paths, key=extract_numeric_part)
total_array=np.array([])

for idx, file_path in enumerate(file_paths):
    
    df= pl.read_csv(file_path)
    value= df.select(pl.sum("value_kwh")) # calculating the total energy consumed in the season
    total_array= np.append(total_array,value)
    if idx%1000 == 0:
        print(f"Total household consumption in {season} for {idx} of {len(file_paths)} done")

x = pl.Series(f"total_{season}_consumption(kWh)",total_array)
min_max_modified_df =min_max_df.hstack([x])
print(min_max_modified_df)
min_max_modified_df.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\min_max_{season}_full.csv')


## Outlier

create a min_max_full.csv file containing the peak and total yearly consumption of the household using the previous code.

In [ ]:
outliers = np.array([])

#min_max_df= pl.read_csv(rf'C:\Users\pana\Desktop\DATA\ckw\min_max__full.csv')
#houses with peak energy consumption of 0 kwh or over 10 kwh or total yearly consumption > 25000 kwh are considered outliers
for i in range(53860):
    if min_max_df[i]["peak"][0] ==0 or min_max_df[i]["peak"][0] > 10 or min_max_df[i]["total_yearly_consmption"][0] > 25000:
        out = i
        outliers= np.append(outliers, out)
    
print(outliers)
outliers_df = pl.DataFrame(outliers)
outliers_df.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\outlier.csv')

## Peak vs Total Yearly Consumption plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize= [8,5])
plt.scatter(df['total_yearly_consmption'], df['peak'],marker = ".", s = 5, label= 'Smart Meter Data')
plt.title(" Peak v/s Total Annual Consumption ")
plt.suptitle(f"Number of records {len(df)}")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel("Total Annual Consumption (kWh)")
plt.ylabel("Peak Consumption (kWh)")